<a href="https://colab.research.google.com/github/GabrieleSocrate/Text-Mining-/blob/main/CreazioneDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# 1. Configurazione Aziende (Microsoft, Apple, Nvidia)
aziende_cik = {
    'Microsoft': '0000789019',
    'Apple': '0000320193',
    'Nvidia': '0001045810'
}

# IMPORTANTE: Inserisci una tua email reale, altrimenti la SEC bloccherà le richieste
headers = {'User-Agent': 'TuoNome TuaEmail@esempio.com'}

# Lista per raccogliere i dataframe di ogni azienda
lista_df = []

print("Recupero degli indici SEC per le aziende...")

# 2. Ciclo per scaricare i metadati di ogni singola azienda
for nome_azienda, cik in aziende_cik.items():
    url_submissions = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"

    response = requests.get(url_submissions, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data['filings']['recent']

        # Creazione DataFrame temporaneo
        df_temp = pd.DataFrame({
            'Azienda': nome_azienda,
            'CIK': cik,
            'Data': filings['filingDate'],
            'Ora': filings['acceptanceDateTime'],
            'Tipo': filings['form'],
            'Accession': filings['accessionNumber'],
            'PrimaryDoc': filings['primaryDocument']
        })

        # 3. Filtriamo 8-K, 10-K, 10-Q e teniamo SOLO I PRIMI 5 per questa azienda
        df_filtrato = df_temp[df_temp['Tipo'].isin(['8-K', '10-K', '10-Q'])].head(5).copy()
        lista_df.append(df_filtrato)
        print(f"- Trovati i documenti per {nome_azienda}")
    else:
        print(f"Errore {response.status_code} per {nome_azienda}")

    # Pausa di cortesia tra una richiesta JSON e l'altra
    time.sleep(0.2)

# Uniamo tutti i risultati in un unico grande DataFrame (15 righe in totale)
df_articoli = pd.concat(lista_df, ignore_index=True)

# 4. Funzione per scaricare e pulire il testo
def download_sec_text(cik, accession, primary_doc):
    time.sleep(0.2) # Pausa obbligatoria per non sovraccaricare il server

    accession_clean = accession.replace('-', '')
    cik_short = str(int(cik))
    url_doc = f"https://www.sec.gov/Archives/edgar/data/{cik_short}/{accession_clean}/{primary_doc}"

    try:
        res = requests.get(url_doc, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, 'html.parser')

            # Pulizia HTML e metadati XBRL
            for tag in soup(["script", "style"]):
                tag.decompose()
            for ix_tag in soup.find_all(['ix:header', 'ix:hidden']):
                ix_tag.decompose()
            for hidden_tag in soup.find_all(style=lambda value: value and 'display:none' in value.replace(' ', '')):
                hidden_tag.decompose()

            return soup.get_text(separator=' ', strip=True)
        else:
            return f"Errore HTTP {res.status_code}"
    except Exception as e:
        return f"Errore nel download: {e}"

print("\nDownload e pulizia in corso dei 15 documenti testuali... (potrebbe volerci un po')")

# 5. Applichiamo la funzione, passando il CIK specifico per ogni riga
df_articoli['Articolo'] = df_articoli.apply(
    lambda row: download_sec_text(row['CIK'], row['Accession'], row['PrimaryDoc']), axis=1
)

# 6. Pulizia finale della colonna Ora e rimozione colonne tecniche inutili
df_articoli['Ora'] = pd.to_datetime(df_articoli['Ora']).dt.time
df_finale = df_articoli[['Azienda', 'Tipo', 'Data', 'Ora', 'Articolo']]

# Mostriamo il risultato finale
print(df_finale)

# Aggiungi questo alla fine del tuo codice Fase 1
df_finale.to_csv("fase1_testi_sec.csv", index=False)

Recupero degli indici SEC per le aziende...
- Trovati i documenti per Microsoft
- Trovati i documenti per Apple
- Trovati i documenti per Nvidia

Download e pulizia in corso dei 15 documenti testuali... (potrebbe volerci un po')
      Azienda  Tipo        Data       Ora  \
0   Microsoft  10-Q  2026-01-28  21:07:34   
1   Microsoft   8-K  2026-01-28  21:04:38   
2   Microsoft   8-K  2025-12-08  21:00:35   
3   Microsoft  10-Q  2025-10-29  20:10:48   
4   Microsoft   8-K  2025-10-29  20:07:42   
5       Apple  10-Q  2026-01-30  11:01:32   
6       Apple   8-K  2026-01-29  21:30:33   
7       Apple   8-K  2026-01-02  21:30:52   
8       Apple   8-K  2025-12-05  21:31:42   
9       Apple  10-K  2025-10-31  10:01:26   
10     Nvidia   8-K  2026-01-23  22:01:33   
11     Nvidia  10-Q  2025-11-19  21:36:17   
12     Nvidia   8-K  2025-11-19  21:20:44   
13     Nvidia  10-Q  2025-08-27  20:52:07   
14     Nvidia   8-K  2025-08-27  20:22:14   

                                             Artic

In [12]:
# 1. Impostazioni: scegli quale articolo e quante parole vuoi vedere
indice_articolo = 0  # 0 è il primo articolo (Microsoft), 1 è il secondo, ecc.
numero_parole = 1000  # Modifica questo numero con le parole che desideri

# 2. Estraiamo il testo completo della riga scelta
testo_completo = df_finale['Articolo'].iloc[indice_articolo]

# 3. Dividiamo il testo in parole, prendiamo le prime 'X' e le riuniamo con uno spazio
parole_estratte = ' '.join(testo_completo.split()[:numero_parole])

# 4. Stampiamo il risultato
print(f"\n--- Prime {numero_parole} parole dell'articolo {indice_articolo} ({df_finale['Azienda'].iloc[indice_articolo]} - {df_finale['Tipo'].iloc[indice_articolo]}) ---")
print(parole_estratte)
print("... [CONTINUA]")


--- Prime 1000 parole dell'articolo 0 (Microsoft - 10-Q) ---
10-Q UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10- Q ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Quarterly Period Ended December 31, 2025 OR ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Transition Period From to Commission File Number 001-37845 MICROSOFT CORPORATION Washington 91-1144442 (STATE OF INCORPORATION) (I.R.S. ID) ONE MICROSOFT WAY , REDMOND , Washington 98052-6399 ( 425 ) 882-8080 www.microsoft.com/investor Securities registered pursuant to Section 12(b) of the Act: Title of each class Trading Symbol Name of exchange on which registered Common stock, $0.00000625 par value per share MSFT Nasdaq 3.125% Notes due 2028 MSFT Nasdaq 2.625% Notes due 2033 MSFT Nasdaq Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d)

In [14]:
import requests
import pandas as pd
import time
import os
from datetime import timedelta

# --- CONFIGURAZIONI FASE 2 ---
POLYGON_API_KEY = "IOfA_Np3cXfnuFJE_fzI3gMmoxdsT39p" # Inserisci la tua chiave API qui
FILE_INPUT = "fase1_testi_sec.csv"         # Il file generato dal tuo codice Fase 1
FILE_OUTPUT = "dataset_finale_completo.csv"

# Il tuo codice Fase 1 ha salvato il nome dell'azienda, ma a Polygon serve il Ticker.
# Creiamo un dizionario per fare la conversione inversa:
mappa_ticker = {
    'Microsoft': 'MSFT',
    'Apple': 'AAPL',
    'Nvidia': 'NVDA'
}

# --- FUNZIONE POLYGON (LENTA MA PRECISA) ---
def ottieni_prezzi_polygon(ticker, data_str, ora_str):
    time.sleep(13) # Rispetto rigoroso del limite gratuito (5 richieste al minuto)
    try:
        # Ricostruiamo la data e l'ora esatta in UTC (come fornite dalla SEC)
        dt_utc = pd.to_datetime(f"{data_str} {ora_str}").tz_localize('UTC')
        # Convertiamo nel fuso orario di Wall Street
        dt_pub = dt_utc.tz_convert('America/New_York')

        start_date = dt_pub.strftime('%Y-%m-%d')
        end_date = (dt_pub + timedelta(days=4)).strftime('%Y-%m-%d') # Finestra per superare i weekend

        url_poly = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&apiKey={POLYGON_API_KEY}"
        res_poly = requests.get(url_poly)

        if res_poly.status_code != 200: return "Err API", None, None

        data_poly = res_poly.json()
        if 'results' not in data_poly or len(data_poly['results']) == 0: return "No Dati", None, None

        df_prezzi = pd.DataFrame(data_poly['results'])
        df_prezzi['datetime'] = pd.to_datetime(df_prezzi['t'], unit='ms', utc=True).dt.tz_convert('America/New_York')

        # Filtriamo per trovare la prima apertura di mercato disponibile
        df_successivo = df_prezzi[df_prezzi['datetime'] >= dt_pub]
        if df_successivo.empty: return "Mercato Chiuso", None, None

        t0_time = df_successivo['datetime'].iloc[0]
        prezzo_t0 = float(df_successivo['c'].iloc[0])

        # Calcoliamo la mezz'ora dopo
        t30_target = t0_time + timedelta(minutes=30)
        df_t30 = df_successivo[df_successivo['datetime'] >= t30_target]
        prezzo_t30 = float(df_successivo['c'].iloc[-1]) if df_t30.empty else float(df_t30['c'].iloc[0])

        delta_perc = ((prezzo_t30 - prezzo_t0) / prezzo_t0) * 100

        # Restituiamo anche l'orario effettivo di mercato in cui è stata calcolata l'apertura
        return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_perc, 2), t0_time.strftime('%Y-%m-%d %H:%M:%S ET')
    except Exception as e:
        return f"Err: {e}", None, None, None

# --- ESECUZIONE PRINCIPALE ---
if not os.path.exists(FILE_INPUT):
    print(f"Errore: Non trovo il file {FILE_INPUT}. Esegui prima il codice Fase 1 e assicurati di aver aggiunto df_finale.to_csv(...) alla fine.")
    exit()

df_testi = pd.read_csv(FILE_INPUT)

# GESTIONE RIPRESA (RESUME)
# Poiché non abbiamo l'ID 'Accession', usiamo la combinazione univoca di Azienda+Data+Ora per capire se un articolo è già stato fatto.
documenti_fatti = set()
if os.path.exists(FILE_OUTPUT):
    df_esistente = pd.read_csv(FILE_OUTPUT)
    # Creiamo una chiave fittizia per identificare la riga
    documenti_fatti = set(df_esistente['Azienda'] + df_esistente['Data_SEC'] + df_esistente['Ora_SEC'])
    print(f"-> Trovati {len(documenti_fatti)} articoli già processati. Riprendo da dove mi ero fermato...")

print(f"Inizio calcolo prezzi. (Premi Ctrl+C nel terminale per fermare il processo e salvare)\n")

for index, row in df_testi.iterrows():
    chiave_univoca = row['Azienda'] + str(row['Data']) + str(row['Ora'])

    if chiave_univoca in documenti_fatti:
        continue # Salta questo articolo se è già nel file di output

    try:
        ticker = mappa_ticker[row['Azienda']]
        print(f"Elaboro prezzi per {row['Azienda']} del {row['Data']} alle {row['Ora']}...")

        # Interroga Polygon
        p_t0, p_t30, delta, ora_mercato = ottieni_prezzi_polygon(ticker, str(row['Data']), str(row['Ora']))

        # Crea la riga completa con i dati testuali inglesi e i nuovi dati finanziari
        nuova_riga = pd.DataFrame({
            'Azienda': [row['Azienda']],
            'Ticker': [ticker],
            'Tipo_Documento': [row['Tipo']],
            'Data_SEC': [row['Data']],
            'Ora_SEC': [row['Ora']],
            'Ora_Effettiva_Mercato': [ora_mercato],
            'Prezzo_T0': [p_t0],
            'Prezzo_T30': [p_t30],
            'Delta_%': [delta],
            'Articolo_EN': [row['Articolo']] # Il dataset testuale rimane intatto
        })

        # Salva in APPEND
        include_header = not os.path.exists(FILE_OUTPUT)
        nuova_riga.to_csv(FILE_OUTPUT, mode='a', index=False, header=include_header)

    except KeyboardInterrupt:
        print("\n[!] Interruzione manuale. Il processo si è fermato in sicurezza. Dati salvati.")
        break
    except Exception as e:
        print(f"Errore strano: {e}")
        continue

print("\nElaborazione terminata! Controlla il file:", FILE_OUTPUT)

Inizio calcolo prezzi. (Premi Ctrl+C nel terminale per fermare il processo e salvare)

Elaboro prezzi per Microsoft del 2026-01-28 alle 21:07:34...
Elaboro prezzi per Microsoft del 2026-01-28 alle 21:04:38...
Elaboro prezzi per Microsoft del 2025-12-08 alle 21:00:35...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:10:48...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:07:42...
Elaboro prezzi per Apple del 2026-01-30 alle 11:01:32...
Elaboro prezzi per Apple del 2026-01-29 alle 21:30:33...
Elaboro prezzi per Apple del 2026-01-02 alle 21:30:52...
Elaboro prezzi per Apple del 2025-12-05 alle 21:31:42...
Elaboro prezzi per Apple del 2025-10-31 alle 10:01:26...
Elaboro prezzi per Nvidia del 2026-01-23 alle 22:01:33...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:36:17...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:20:44...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:52:07...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:22:14...

Elaborazione terminata! Controll

In [15]:
import pandas as pd

# 1. Carichiamo il file appena creato
df_verifica = pd.read_csv("dataset_finale_completo.csv")

# 2. Stampiamo un riepilogo generale (quante righe, quante colonne)
print("=== INFO GENERALI DATASET ===")
print(df_verifica.info())
print("-" * 50)

# 3. Controlliamo se i calcoli finanziari hanno senso
# Mostriamo solo le colonne chiave per le prime 5 righe
print("\n=== VERIFICA PREZZI E DELTA (Prime 5 righe) ===")
colonne_finanza = ['Azienda', 'Tipo_Documento', 'Data_SEC', 'Prezzo_T0', 'Prezzo_T30', 'Delta_%']
print(df_verifica[colonne_finanza].head())
print("-" * 50)

# 4. Controlliamo la purezza del testo per il text mining
# Prendiamo le prime 300 parole del primo articolo per assicurarci che sia in inglese e senza codice HTML
print("\n=== VERIFICA TESTO (Estratto primo documento) ===")
primo_articolo = str(df_verifica['Articolo_EN'].iloc[0])
estratto_parole = ' '.join(primo_articolo.split()[:50]) # Prende le prime 50 parole
print(estratto_parole + " [...]")


=== INFO GENERALI DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Azienda                15 non-null     object 
 1   Ticker                 15 non-null     object 
 2   Tipo_Documento         15 non-null     object 
 3   Data_SEC               15 non-null     object 
 4   Ora_SEC                15 non-null     object 
 5   Ora_Effettiva_Mercato  15 non-null     object 
 6   Prezzo_T0              15 non-null     float64
 7   Prezzo_T30             15 non-null     float64
 8   Delta_%                15 non-null     float64
 9   Articolo_EN            15 non-null     object 
dtypes: float64(3), object(7)
memory usage: 1.3+ KB
None
--------------------------------------------------

=== VERIFICA PREZZI E DELTA (Prime 5 righe) ===
     Azienda Tipo_Documento    Data_SEC  Prezzo_T0  Prezzo_T30  Delta_%
0  Microsoft  